In [ ]:
# ? LIBRERIAS

import os
import arcpy
import pandas as pd
import numpy as np
import csv
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import errno
import pathlib

arcpy.env.overwriteOutput = True

In [ ]:
# ** Se crea dataframe que almacenará los resultados de la consulta
columnas_df_resultados = ['ruta_archivo','nombre_archivo']
df_resultados_lectura_numeros_pagina = pd.DataFrame(columns = columnas_df_resultados)

columnas_df_archivo_inoperables = ['ruta_arhivo']
df_archivo_inoperables = pd.DataFrame(columns = columnas_df_archivo_inoperables)

In [ ]:
ruta_directorio_insumos = r"D:\PUBLIC\PGAR\Insumos\Informacion_NO_Estructurada"

# ** Se accede a todos los archivos de una carpeta en específico
lista_rutas = []
lista_archivos = []
lista_archivo_inoperables = []
              
try:
    for ruta, directorio, archivo in os.walk(ruta_directorio_insumos):
        # ** Se navega por cada uno de los archivos para, posteriormente, generar un listado de rutas
        for nombre_archivo in archivo:
            # ** Se usa la funcion PdfReader para leer el PDF asociado a la ruta incorporada
            ruta_archivo = os.path.join(ruta, nombre_archivo)
            # ? Se utiliza, del módulo os, la función stat y el atributo st_size para encontrar los pesos de los archivos
            # ? y eliminar del análisis los pdf corruptos
            sizefile = os.stat(ruta_archivo).st_size
            if (pathlib.Path(ruta_archivo).suffix == '.xlsx' or pathlib.Path(ruta_archivo).suffix == '.xls' or pathlib.Path(ruta_archivo).suffix == '.csv') and sizefile != 0:
                lista_rutas.append(ruta_archivo)
                lista_archivos.append(nombre_archivo)
            elif (pathlib.Path(ruta_archivo).suffix == '.xlsx' or pathlib.Path(ruta_archivo).suffix == '.xls' or pathlib.Path(ruta_archivo).suffix == '.csv') and sizefile == 0:
                lista_archivo_inoperables.append(ruta_archivo)
                print("El archivo {0} es inoperable".format(nombre_archivo))
            else:
                print("{0} no es un archivo XLSX o CSV o XLS".format(ruta_archivo))
                    
except OSError as e:
    if e.errno != errno.EEXIST:
        raise  

In [ ]:
# ** Asignación de listas a dataframe
df_resultados_lectura_numeros_pagina['ruta_archivo'] = lista_rutas
df_resultados_lectura_numeros_pagina['nombre_archivo'] = lista_archivos

df_archivo_inoperables['ruta_archivo'] = lista_archivo_inoperables

In [ ]:
# ** Generación de CSV de salida
df_resultados_lectura_numeros_pagina.to_csv("inventario_archivos.csv", index = False, sep = ";")

df_archivo_inoperables.to_csv("descripcion_inoperables.csv", index = False, sep = ";")